# Part a) 
- load and prepare Oxford Pets Data

In [1]:
import pytorch_lightning as pl
import torch
import torchvision
from pathlib import Path
import random
import torchvision.transforms.functional as F

class RandomRotation(object):
    def __init__(self, degrees, seed=1):
        self.degrees = (-degrees, degrees)
        random.seed(seed)
    
    @staticmethod
    def get_params(degrees):
        angle = random.uniform(degrees[0], degrees[1])
        return angle

    def __call__(self, img):
        angle = self.get_params(self.degrees)
        return F.rotate(img, angle)

In [2]:
from torchvision import transforms
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]


train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

ds_train = torchvision.datasets.OxfordIIITPet(
    root=".", 
    split="trainval", 
    transform=train_transforms, 
    download=True
)

ds_test = torchvision.datasets.OxfordIIITPet(
    root=".", 
    split="test", 
    transform=test_transforms, 
    download=True
)

ds_train_normal = torchvision.datasets.OxfordIIITPet(
    root=".", 
    split="trainval", 
    transform=test_transforms, 
    download=True
)

BATCH_SIZE = 512

dl_train = torch.utils.data.DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

  0%|          | 0/791918971 [00:00<?, ?it/s]

Extracting oxford-iiit-pet/images.tar.gz to oxford-iiit-pet


  0%|          | 0/19173078 [00:00<?, ?it/s]

Extracting oxford-iiit-pet/annotations.tar.gz to oxford-iiit-pet


# Part b and c)
- define and train a simple CNN
- use skip connections and batch normalization


# DISCLAIMER: The Test Set accuracy is evaluated all at once at the bottom of this document to make it easier to compare them.

In [3]:
import pytorch_lightning as pl
from typing import Any
import torchmetrics
import torch

class CnnWrapper(pl.LightningModule):

    def __init__(self, 
        loss: callable, 
        lr: float, 
        architecture: torch.nn.Module, 
        classification_head:torch.nn.Module,
        num_classes:int
    ) -> None:
        super().__init__()
        self.architecture = architecture
        self.classification_head = classification_head 
        self.num_classes = num_classes
        self.loss = loss
        self.lr = lr
        
        self.test_auroc = torchmetrics.AUROC(num_classes=self.num_classes)
        self.test_acc = torchmetrics.Accuracy(num_classes=self.num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.architecture(x)
        if self.classification_head is not None:
            x = self.classification_head(x)
        return x

    def _step(self, batch) -> torch.Tensor:
        x, y = batch
        pred = self.forward(x)
        loss = self.loss(pred, y)
        return pred, loss

    def training_step(self, batch) -> torch.Tensor:
        pred, loss = self._step(batch)
        self.log("train/loss", loss)
        pred = torch.nn.functional.softmax(pred, dim=1)
        acc = torchmetrics.functional.accuracy(pred, batch[-1], num_classes=self.num_classes)
        self.log("train/acc", acc)
        return loss
    
    def _eval_step(self, batch, auroc, acc):
        pred, loss = self._step(batch)
        pred = torch.nn.functional.softmax(pred, dim=1)
        auroc.update(pred, batch[-1])
        acc.update(pred, batch[-1])
        return loss
        
    def test_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        loss = self._eval_step(batch, self.test_auroc, self.test_acc)
        self.log("test", loss)

    def test_epoch_end(self, outputs) -> None:
        print(f"Test AUROC: {self.test_auroc.compute().data}")
        print(f"Test Accuracy: {self.test_acc.compute().data}")

    def validation_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        loss = self._eval_step(batch, self.test_auroc, self.test_acc)
        self.log("test", loss)

    def validation_epoch_end(self, outputs) -> None:
        print(f"Val AUROC: {self.test_auroc.compute().data}")
        print(f"Val Accuracy: {self.test_acc.compute().data}")

        
    def configure_optimizers(self) -> Any:
        optim = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optim

In [4]:
simple_architecture = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, 3),
    torch.nn.MaxPool2d((4, 4)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 32, 3),
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 40, 3),
    torch.nn.ReLU(),
    torch.nn.Conv2d(40, 48, 3),
    torch.nn.ReLU()
)

class SkipBlock(torch.nn.Module):

    def __init__(self, in_channels) -> None:
        super().__init__()
        self.in_chanels = in_channels
        self.block = torch.nn.Sequential(
            torch.nn.Conv2d(self.in_chanels, self.in_chanels, kernel_size=(3, 3), padding="same"),
            torch.nn.BatchNorm2d(self.in_chanels),
            torch.nn.ReLU(),
            torch.nn.Conv2d(self.in_chanels, self.in_chanels, kernel_size=(3, 3), padding="same"),
            torch.nn.BatchNorm2d(self.in_chanels),
        )

    def forward(self, x):
        id = x
        out = self.block(x)
        out += id
        return torch.nn.functional.relu(out)

better_architecture = torch.nn.Sequential(
    torch.nn.Conv2d(3, 8, 3), 
    torch.nn.BatchNorm2d(8),
    torch.nn.ReLU(),
    SkipBlock(8),
    torch.nn.Conv2d(8, 16, (3, 3)),
    torch.nn.MaxPool2d((4, 4)),
    SkipBlock(16),
    SkipBlock(16),
    torch.nn.Conv2d(16, 32, (3, 3)),
    torch.nn.MaxPool2d((2, 2)),
    SkipBlock(32),
    SkipBlock(32),
    torch.nn.Conv2d(32, 40, (3, 3)),
    torch.nn.Dropout(),
    SkipBlock(40),
    SkipBlock(40),    
    torch.nn.Conv2d(40, 48, (3, 3)),
    torch.nn.Dropout(),
    SkipBlock(48),
    SkipBlock(48),
)

In [5]:
num_classes = len(ds_train.classes)
loss = torch.nn.CrossEntropyLoss()
lr = 1e-3
epochs = 150

In [6]:
classification_head = torch.nn.Sequential(
    torch.nn.AdaptiveAvgPool2d((1, 1)),
    torch.nn.Flatten(),
    torch.nn.Linear(48, num_classes),
)

simple_model = CnnWrapper(loss, lr, simple_architecture, classification_head, num_classes)
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=epochs, log_every_n_steps=10, check_val_every_n_epoch=5)

# i use the test set as the validation set, to get a feeling of how the network is performing, but no actions are taken on it so it should be fine
# this comment also applies to every following network training loop
trainer.fit(simple_model, dl_train, val_dataloaders=[dl_test])

/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Sanity Checking: 0it [00:00, ?it/s]

Val AUROC: 0.1438794732093811
Val Accuracy: 0.0380859375


/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1896: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

Val AUROC: 0.7334133982658386
Val Accuracy: 0.11859554797410965


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.7351413369178772
Val Accuracy: 0.11981382220983505


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.7367457151412964
Val Accuracy: 0.12096063047647476


In [7]:
classification_head = torch.nn.Sequential(
    torch.nn.AdaptiveAvgPool2d((1, 1)),
    torch.nn.Flatten(),
    torch.nn.Linear(48, num_classes),
)

better_model = CnnWrapper(loss, lr, better_architecture, classification_head, num_classes)
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=epochs, log_every_n_steps=10, check_val_every_n_epoch=5)

trainer.fit(better_model, dl_train, val_dataloaders=[dl_test])

/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Sanity Checking: 0it [00:00, ?it/s]

Val AUROC: 0.13891097903251648
Val Accuracy: 0.0029296875


/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1896: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

Val AUROC: 0.833551287651062
Val Accuracy: 0.23701761662960052


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.8365272879600525
Val Accuracy: 0.2414056360721588


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.8384901881217957
Val Accuracy: 0.24495472013950348


# Part d mixup

- Mixup just modifes the data, so we can just write a custom Dataset that does the mixup
- We do not need to change network interners since CrossEntropyCost works in multiclass classification

In [8]:
from typing import Tuple
import torch
import numpy as np

class MixupPets(torch.utils.data.Dataset):

    def __init__(self, pets_ds: torch.utils.data.Dataset) -> None:
        super().__init__()
        self.ds_pets = pets_ds
        self.num_classes = len(pets_ds.classes)
        
    def __len__(self) -> int:
        return len(self.ds_pets)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        x, y = self.ds_pets[idx]

        l = torch.zeros(self.num_classes)
        l[y] = 1.

        rv = random.randint(0, self.__len__() - 1)
        x_m, y_m = self.ds_pets[rv]
        l_m = torch.zeros(self.num_classes)
        l_m[y_m] = 1

        alpha = 0.2
        lam = np.random.beta(alpha, alpha)
        image = lam * x + (1 - lam) * x_m
        label = lam * l + (1 - lam) * l_m

        return image, label


In [9]:
ds_mixup = MixupPets(ds_train_normal)
dl_mixup = torch.utils.data.DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

classification_head = torch.nn.Sequential(
    torch.nn.AdaptiveAvgPool2d((1, 1)),
    torch.nn.Flatten(),
    torch.nn.Linear(48, num_classes),
)

mixup_model = CnnWrapper(loss, lr, better_architecture, classification_head, num_classes)
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=epochs, log_every_n_steps=10, check_val_every_n_epoch=5)

trainer.fit(mixup_model, dl_mixup, val_dataloaders=[dl_test])

Sanity Checking: 0it [00:00, ?it/s]

Val AUROC: 0.14747370779514313
Val Accuracy: 0.0


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Val AUROC: 0.7042688131332397
Val Accuracy: 0.09716599434614182


...

Val AUROC: 0.9047836065292358
Val Accuracy: 0.38078761100769043


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.9058112502098083
Val Accuracy: 0.38312309980392456


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.9059702157974243
Val Accuracy: 0.38417917490005493


# Part e)

- Install timm and torchinfo to get a feeling of how the loaded model looks
- Freeze model excluding the classification head
- Finetune

In [10]:
# !pip install timm
# !pip install torchinfo==1.6.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 2.5 MB/s eta 0:00:00


In [11]:
from torchinfo import summary
import timm

#timm.list_models()
timm_model = timm.create_model('resnest50d', pretrained=True, num_classes=num_classes)
# inspect the loaded model to get a feeling for how it looks and what to change / freeze
summary(timm_model, input_size = (BATCH_SIZE, 3, 224, 224))


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-resnest/resnest50-528c19ca.pth" to /root/.cache/torch/hub/checkpoints/resnest50-528c19ca.pth


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Sequential: 1-1                        [512, 64, 112, 112]       --
│    └─Conv2d: 2-1                       [512, 32, 112, 112]       864
│    └─BatchNorm2d: 2-2                  [512, 32, 112, 112]       64
│    └─ReLU: 2-3                         [512, 32, 112, 112]       --
│    └─Conv2d: 2-4                       [512, 32, 112, 112]       9,216
│    └─BatchNorm2d: 2-5                  [512, 32, 112, 112]       64
│    └─ReLU: 2-6                         [512, 32, 112, 112]       --
│    └─Conv2d: 2-7                       [512, 64, 112, 112]       18,432
├─BatchNorm2d: 1-2                       [512, 64, 112, 112]       128
├─ReLU: 1-3                              [512, 64, 112, 112]       --
├─MaxPool2d: 1-4                         [512, 64, 56, 56]         --
├─Sequential: 1-5                        [512, 256, 56, 56]        --
│    └

In [12]:
# freeze model layers that don't need finetuning
def freeze_timm_model(model):
    '''Freeze all layers except the last layer(fc or classifier)'''
    for param in model.parameters():
            param.requires_grad = False
    # reset the last two layers to require grad
    model.fc.weight.requires_grad = True
    model.fc.bias.requires_grad = True
    
freeze_timm_model(timm_model)

In [13]:
# Finetune the network
epochs = 30 # we don't need as many epochs 
pretrained_model = CnnWrapper(loss, lr, timm_model, None, num_classes)
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=epochs, log_every_n_steps=5, check_val_every_n_epoch=5)

trainer.fit(pretrained_model, dl_train, val_dataloaders=[dl_test])

Sanity Checking: 0it [00:00, ?it/s]

Val AUROC: 0.18190491199493408
Val Accuracy: 0.0927734375


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Val AUROC: 0.9670823812484741
Val Accuracy: 0.679096519947052


...

Val AUROC: 0.9896060228347778
Val Accuracy: 0.818980872631073


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.9913065433502197
Val Accuracy: 0.8313283920288086


Validation: 0it [00:00, ?it/s]

Val AUROC: 0.9925006628036499
Val Accuracy: 0.8409150242805481


# Run the Test Step for every Network to see how the performance differs

In [14]:
print(f"Testing Simple CNN")
trainer.test(simple_model, dl_test)
print(f"Testing Better CNN")
trainer.test(better_model, dl_test)
print(f"Testing Better CNN with Mixup")
trainer.test(mixup_model, dl_test)
print(f"Testing Finetuned Resnet50d")
trainer.test(pretrained_model, dl_test)

Testing Simple CNN


Testing: 0it [00:00, ?it/s]

Test AUROC: 0.738244891166687
Test Accuracy: 0.12203410267829895


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           test            │    3.1024057865142822     │
└───────────────────────────┴───────────────────────────┘

Testing Better CNN


...

Testing Finetuned Resnet50d


Testing: 0it [00:00, ?it/s]

Test AUROC: 0.9933205842971802
Test Accuracy: 0.8478676080703735


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           test            │    0.3644622564315796     │
└───────────────────────────┴───────────────────────────┘

[{'test': 0.3644622564315796}]

# Findings:
- The performance of the networks increases with each subtask -> New Architectures + model training methods improve performance
- Models probably overfited a little bit, the performance of the first 3 models was better in a test run with only ~80 epochs. 
- Finetunig a Pretrained Model is way less expansive and often performs better than handcrafted models 
    - Due to training set size, computing power, etc